<a href="https://colab.research.google.com/github/deea-c/thesis_absa/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #To install
!pip install evaluate
!pip install seqeval (need to install this) 
!pip install optuna
!pip install transformers
!pip install seqeval
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.8 MB/s eta 0:00:00
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip inst

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/code
!pwd 

/content/drive/MyDrive/Colab Notebooks/code
/content/drive/MyDrive/Colab Notebooks/code


In [ ]:
#Packages
from help_functions import get_annotations_as_dict, align_tokens_and_annotations_bio, compute_metrics
from help_functions import metric
from help_functions import TraingDataset, LabelSet, TrainingExample, label_list, label_set
import pandas as pd 
import numpy as np
from transformers import AutoTokenizer,EarlyStoppingCallback
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report
import evaluate
np.random.seed(11)


In [ ]:
language = "English"
model_name = 'xlm-roberta-base'

In [ ]:
# #### Load data
df = get_annotations_as_dict( pd.read_pickle("../pickles/" + "train_"+language +".pkl"))
test_df = get_annotations_as_dict( pd.read_pickle("../pickles/" + "test_"+language + ".pkl"))

In [ ]:
df.head()

,Sentence_id,Text,annotations,polarities
0,1004293:0,Judging from previous posts this used to be a ...,"[{'Opinion_target': 'place', 'Opinion_category...",negative
1,1004293:1,"We, there were four of us, arrived at noon - t...","[{'Opinion_target': 'staff', 'Opinion_category...",negative
2,1004293:2,"They never brought us complimentary noodles, i...","[{'Opinion_target': 'NULL', 'Opinion_category'...",negative
3,1004293:3,The food was lousy - too sweet or too salty an...,"[{'Opinion_target': 'food', 'Opinion_category'...","negative,negative"
4,1004293:4,"After all that, they complained to me about th...","[{'Opinion_target': 'NULL', 'Opinion_category'...",negative


In [ ]:
# #Train & Test data read with multile datasets
# df1 = pd.read_pickle("../pickles/" + "train_"+"English" +".pkl")
# df2 = pd.read_pickle("../pickles/" + "train_"+"French" +".pkl")
# df = pd.concat([df1, df2], axis=0)
# df.reset_index(drop=True, inplace=True)
# df = get_annotations_as_dict(df)

# tdf1 = pd.read_pickle("../pickles/" + "test_"+"English" +".pkl")
# tdf2 = pd.read_pickle("../pickles/" + "test_"+"French" +".pkl")
# test_df =  pd.concat([tdf1, tdf2], axis=0)
# test_df.reset_index(drop=True, inplace=True)
# test_df = get_annotations_as_dict(test_df)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name) # Load a pre-trained tokenizer
label_set = LabelSet(labels=["positive", "negative", "neutral"])

ds = TraingDataset(
    data=df, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=89)


test_ds = TraingDataset(
    data=test_df, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=89)

data_collator = DataCollatorForTokenClassification(tokenizer)
label_list = label_set.ids_to_label
print(label_list)
train_ds , val_ds= train_test_split(ds, test_size=0.2, random_state=11, shuffle=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_set.labels_to_id), label2id = label_set.labels_to_id, id2label =label_set.ids_to_label )


{0: 'O', 1: 'B-positive', 2: 'I-positive', 3: 'B-negative', 4: 'I-negative', 5: 'B-neutral', 6: 'I-neutral'}


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
# Pass Training Arguments
batch_size = 16
model_n = model_name.split("/")[-1]
args = TrainingArguments(
    output_dir = f"models/{model_n}-{language}-finetuned",
    overwrite_output_dir = True,
    logging_dir= f"models/{model_n}-{language}-finetuned/log",
    evaluation_strategy = "epoch",
    logging_strategy= "epoch",
    num_train_epochs =10, 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    seed = 11,
    push_to_hub= False,
)


In [ ]:
from transformers import TrainerCallback
from copy import deepcopy
class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy


In [ ]:
### Code to add dropout to mbert - Result did not improve
# from transformers import AutoConfig 
# dropout_model_config = model.config
# dropout_model_config                                       
# dropout_model_config.classifier_dropout =0.3
# dropout_model =  AutoModelForTokenClassification.from_pretrained(model_name, config = dropout_model_config)
# dropout_model

In [ ]:

trainer = Trainer(
    model,
    args,
    train_dataset= train_ds,
    eval_dataset= val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )
trainer.add_callback(CustomCallback(trainer))
# logging_callback = 
model_fin =trainer.train()
training_logs = trainer.state.log_history
log_df = pd.DataFrame(training_logs).groupby(['epoch', 'step']).agg(lambda x:  x.dropna().max()).reset_index()
log_df.to_pickle(f"../models/{model_n}-{language}-earlys-log.pkl")
trainer.save_model("../models/"+ f"{model_n}-{language}-earlys")

In [ ]:
log_df[["epoch","loss","eval_loss",'train_val_precision',
       'train_val_recall', 'train_val_f1', 'train_val_accuracy',  'eval_val_precision',
       'eval_val_recall', 'eval_val_f1', 'eval_val_accuracy', ]]



In [ ]:
#plot
import matplotlib.pyplot as plt

def plot_history(data_list, label_list, title, ylabel):
    ''' Plots a list of vectors.
    Parameters:
        data_list  : list of vectors containing the values to plot
        label_list : list of labels describing the data, one per vector
        title      : title of the plot
        ylabel     : label for the y axis
    '''
    epochs = range(1, len(data_list[0]) + 1)
    for data, label in zip(data_list, label_list):
        plt.plot(epochs, data, label=label)
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
%matplotlib inline

acc = log_df['train_val_accuracy']
val_acc = log_df['eval_val_accuracy']
loss = log_df['loss']
val_loss = log_df['eval_loss']

plot_history(data_list=[loss, val_loss],
             label_list=['Training loss', 'Validation loss'],
             title='Training and validation loss',
             ylabel='Loss')
plot_history(data_list=[acc, val_acc],
             label_list=['Training acc.', 'Validation acc.'],
             title='Training and validation accuracy',
             ylabel='Accuracy')

In [ ]:
trainer.evaluate(test_ds)

In [ ]:
predictions, labels, _ =trainer.predict(test_ds)
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)

print(classification_report(true_labels,true_predictions))

In [ ]:
fr_test_df = get_annotations_as_dict( pd.read_pickle("../pickles/" + "test_"+"dutch" + ".pkl"))
fr_test_ds = TraingDataset(
    data=fr_test_df, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=89)
trainer.evaluate(fr_test_ds)

In [ ]:
lang = "English"
path =   f"../models/"+f"{model_n}-{lang}-base" # "./models/"+ f"{model_name}-finetuned"
load_model = AutoModelForTokenClassification.from_pretrained(path)
#Train the model
newtrainer = Trainer(
    load_model,
    # args,
    train_dataset= train_ds,
    eval_dataset= val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

predictions, labels, _ = newtrainer.predict(fr_test_ds)
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels,)
print("test", results, sep="\n" )

In [ ]:
lang = "French"
path =   f"../models/"+f"{model_n}-{lang}-base" # "./models/"+ f"{model_name}-finetuned"
load_model = AutoModelForTokenClassification.from_pretrained(path)
#Train the model
newtrainer = Trainer(
    load_model,
    # args,
    train_dataset= train_ds,
    eval_dataset= val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

predictions, labels, _ = newtrainer.predict(fr_test_ds)
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels,)
print("test", results, sep="\n" )

In [ ]:
trainer.evaluate(fr_test_ds)